HARSHINI NAIDU GANAPATHY 
(G39946084)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.naive_bayes import GaussianNB

/var/folders/7p/11qrtpkd599_xr7bbhj3ms980000gn/T/ipykernel_28289/250016848.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Load and preprocess the dataset
data = pd.read_csv('MUP_DPR_RY23_P04_V10_DY21_Geo.csv')

# Encode 'Gnrc_Name' and combine labels into a single target
encoder = LabelEncoder()
data['Gnrc_Name_Encoded'] = encoder.fit_transform(data['Gnrc_Name'])
conditions = [
    (data['Opioid_Drug_Flag'] == 'Y'),
    (data['Opioid_LA_Drug_Flag'] == 'Y'),
    (data['Antbtc_Drug_Flag'] == 'Y'),
    (data['Antpsyct_Drug_Flag'] == 'Y')
]
choices = ['Opioid', 'Opioid_LA', 'Antibiotic', 'Antipsychotic']
data['Drug_Classification'] = np.select(conditions, choices, default='Other')
data['Drug_Classification_Encoded'] = encoder.fit_transform(data['Drug_Classification'])

# Selecting features and target
features_selected = ['Tot_Clms', 'Tot_Drug_Cst', 'Gnrc_Name_Encoded']
X = data[features_selected].values
y = data['Drug_Classification_Encoded'].values

# Scale features
scaler = StandardScaler() #To make sure all 3 features contribute equally 
X_scaled = scaler.fit_transform(X)

/var/folders/7p/11qrtpkd599_xr7bbhj3ms980000gn/T/ipykernel_28289/4280166366.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('MUP_DPR_RY23_P04_V10_DY21_Geo.csv')


In [4]:
data.head(40)

,Prscrbr_Geo_Lvl,Prscrbr_Geo_Cd,Prscrbr_Geo_Desc,Brnd_Name,Gnrc_Name,Tot_Prscrbrs,Tot_Clms,Tot_30day_Fills,Tot_Drug_Cst,Tot_Benes,...,GE65_Tot_Benes,LIS_Bene_Cst_Shr,NonLIS_Bene_Cst_Shr,Opioid_Drug_Flag,Opioid_LA_Drug_Flag,Antbtc_Drug_Flag,Antpsyct_Drug_Flag,Gnrc_Name_Encoded,Drug_Classification,Drug_Classification_Encoded
0,National,NaN,National,1st Tier Unifine Pentips,"Pen Needle, Diabetic",1178,3708,6598.5,1.022808e+05,1313.0,...,1052.0,4167.66,22903.26,N,N,N,N,1342,Other,3
1,National,NaN,National,1st Tier Unifine Pentips Plus,"Pen Needle, Diabetic",1604,4564,7905.6,1.319273e+05,1766.0,...,1450.0,4415.22,20335.83,N,N,N,N,1342,Other,3
2,National,NaN,National,Abacavir,Abacavir Sulfate,3560,30534,37402.4,7.039482e+06,4252.0,...,2141.0,21278.04,186025.79,N,N,N,N,3,Other,3
3,National,NaN,National,Abacavir-Lamivudine,Abacavir Sulfate/Lamivudine,2933,30248,38810.9,9.656850e+06,4030.0,...,2272.0,23576.53,376349.75,N,N,N,N,4,Other,3
4,National,NaN,National,Abacavir-Lamivudine-Zidovudine,Abacavir/Lamivudine/Zidovudine,38,118,120.0,1.467952e+05,32.0,...,19.0,1488.48,3939.47,N,N,N,N,6,Other,3
5,National,NaN,National,Abelcet,Amphotericin B Lipid Complex,67,697,717.4,5.045453e+05,199.0,...,131.0,947.12,62103.18,N,N,N,N,101,Other,3
6,National,NaN,National,Abilify,Aripiprazole,2512,15246,19657.3,2.046701e+07,2172.0,...,691.0,35948.95,259326.22,N,N,N,Y,118,Antipsychotic,1
7,National,NaN,National,Abilify Maintena,Aripiprazole,15911,208282,209057.7,4.982323e+08,26321.0,...,3360.0,484272.75,1162541.41,N,N,N,Y,118,Antipsychotic,1
8,National,NaN,National,Abilify Mycite,Aripiprazole,6,31,33.0,5.630880e+04,NaN,...,NaN,70.00,60.00,N,N,N,Y,118,Antipsychotic,1
9,National,NaN,National,Abiraterone Acetate,Abiraterone Acetate,10065,199302,203684.4,6.011826e+08,30507.0,...,29429.0,595635.61,52997200.21,N,N,N,N,12,Other,3


In [12]:
# PCA for dimensionality reduction
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
print(f"Explained variance by component: {pca.explained_variance_ratio_}")
print(f"Cumulative Explained variance: {np.cumsum(pca.explained_variance_ratio_)}")
# how much information is retained in the reduced data.
print(f"PCA components: \n{pca.components_}")
#This will give you the vectors in the feature space, indicating the directions of maximum variance (the principal components).


Explained variance by component: [0.41291083 0.3332267 ]
Cumulative Explained variance: [0.41291083 0.74613754]
PCA components: 
[[ 0.70659027  0.70667006 -0.03670992]
 [ 0.02813414  0.02378154  0.99932122]]


In [6]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Initialize and evaluate base models
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
}

X_train_meta = np.zeros((X_train.shape[0], len(models)))  # Placeholder for base model predictions
X_test_meta = np.zeros((X_test.shape[0], len(models)))

for i, (name, model) in enumerate(models.items()):
    # Train model
    model.fit(X_train, y_train)
    
    # Generate predictions for evaluation
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    # Store test predictions for meta model training
    X_train_meta[:, i] = train_predictions
    X_test_meta[:, i] = test_predictions
    
    # Evaluate model
    accuracy = accuracy_score(y_test, test_predictions)
    kappa = cohen_kappa_score(y_test, test_predictions)
    print(f'{name} - Test Accuracy: {accuracy:.4f}, Kappa: {kappa:.4f}')

Random Forest - Test Accuracy: 0.9874, Kappa: 0.9206
Gradient Boosting - Test Accuracy: 0.9589, Kappa: 0.6943
K-Nearest Neighbors - Test Accuracy: 0.9785, Kappa: 0.8671
Decision Tree - Test Accuracy: 0.9884, Kappa: 0.9292


In [8]:
X_train_meta[:20]

array([[3., 3., 3., 3.],
       [3., 3., 3., 3.],
       [3., 3., 3., 3.],
       [3., 3., 3., 3.],
       [3., 3., 3., 3.],
       [3., 3., 3., 3.],
       [1., 1., 1., 1.],
       [3., 3., 3., 3.],
       [3., 3., 3., 3.],
       [0., 0., 0., 0.],
       [3., 3., 3., 3.],
       [3., 3., 3., 3.],
       [3., 3., 3., 3.],
       [3., 3., 3., 3.],
       [3., 3., 3., 3.],
       [3., 3., 3., 3.],
       [3., 3., 3., 3.],
       [3., 3., 3., 3.],
       [3., 3., 3., 3.],
       [3., 3., 3., 3.]])

In [11]:
# Train the meta model using base model predictions on the training set
meta_model = GaussianNB()
meta_model.fit(X_train_meta, y_train)

# Final predictions from the meta model using test set predictions of base models
final_predictions = meta_model.predict(X_test_meta)

# Evaluate the ensemble model
ensemble_accuracy = accuracy_score(y_test, final_predictions)
ensemble_kappa = cohen_kappa_score(y_test, final_predictions)
print(f'Ensemble model - Accuracy: {ensemble_accuracy:.4f}, Kappa: {ensemble_kappa:.4f}')


Ensemble model - Accuracy: 0.9873, Kappa: 0.9218


In [15]:
#Application
#Pharmacy Inventory Management

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load your dataset
file_path = 'MUP_DPR_RY23_P04_V10_DY21_Geo.csv'  # Update this path
data = pd.read_csv(file_path)

# Assuming 'Prscrbr_Geo_Desc' as the geographical identifier and 'Tot_Clms' as the demand
# Let's aggregate total claims by 'Prscrbr_Geo_Desc'
# For a real project, you would also incorporate time and specific drugs into this aggregation
data_aggregated = data.groupby('Prscrbr_Geo_Desc').agg({'Tot_Clms': 'sum'}).reset_index()

# Simulate additional features for each geographical area
# In a real scenario, these could be demographics, healthcare facility counts, etc.
np.random.seed(42)
data_aggregated['Population'] = np.random.randint(1000, 100000, size=data_aggregated.shape[0])
data_aggregated['Healthcare_Facilities'] = np.random.randint(1, 20, size=data_aggregated.shape[0])

# Preparing the data for modeling
X = data_aggregated[['Population', 'Healthcare_Facilities']]
y = data_aggregated['Tot_Clms']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeling - using a RandomForestRegressor as an example
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions and evaluation
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)

print(f"Root Mean Square Error: {rmse}")

# Continuing from the previous aggregation and assuming 'data_aggregated' is your DataFrame


# Sort the data by 'Tot_Clms' in descending order to find highest demand
data_sorted = data_aggregated.sort_values(by='Tot_Clms', ascending=False)


# Display the top locations with the highest demand for prescriptions
top_locations = data_sorted.head(10)  # Adjust the number to display as needed


print("Top Locations with High Demand for Prescriptions:")
print(top_locations[['Prscrbr_Geo_Desc', 'Tot_Clms']])



Root Mean Square Error: 259887929.3851866
Top Locations with High Demand for Prescriptions:
   Prscrbr_Geo_Desc    Tot_Clms
33         National  1500037987
8        California   136761073
13          Florida   113926838
52            Texas    98437767
39         New York    97559210
46     Pennsylvania    73985161
43             Ohio    63310918
40   North Carolina    53047057
19         Illinois    51880804
28         Michigan    50859359


/var/folders/7p/11qrtpkd599_xr7bbhj3ms980000gn/T/ipykernel_58634/3038041089.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)
